# EVALUATION LOOP

In [1]:
# ------------------------------------------------------------
# 0.  Imports & config – nothing here should clash with yours
# ------------------------------------------------------------
from tqdm import tqdm
import torch
from mathbert_encoder import MathBERTEncoder
import retriever_cosine as rc
# from import retrieve_top_k_cosine, retrieve_sample_k_cosine
from response_sampler import sample_responses_per_demo
from reward_aggregator import compute_demo_accuracy
from icl_model_wrapper import OpenAIICLModel, OpenAIAdvanced
from grpo_optimizer import grpo_step
from datasets import load_dataset
from dotenv import load_dotenv
import os
from transformers import get_linear_schedule_with_warmup
from importlib import reload
import helpers

reload(rc)

load_dotenv()

API_KEY = os.getenv("OPENAI_API_KEY")
DEVICE  = "cuda" if torch.cuda.is_available() else "cpu"
K       = 1             # demos per query
NUM_SAMPLES = 1           # model completions per query
TEMPERATURE = 0.7           # keep same as training loop

# ------------------------------------------------------------
# 1.  Initialise encoder **in eval mode** (weights frozen)
# ------------------------------------------------------------
encoder = MathBERTEncoder(device=DEVICE, trainable=False)
encoder.eval()                                   # no grads!

icl_model = OpenAIAdvanced(api_key=API_KEY,
                           model_name="gpt-4.1-nano",
                           temperature=TEMPERATURE)

# ------------------------------------------------------------
# 2.  Load/define the inference set
#     If you already saved a slice elsewhere, just load it.
# ------------------------------------------------------------
train_path = "./data/MATH/train.jsonl"
train_ds_list = helpers.load_jsonl(train_path)[:40]
raw_demos = helpers.load_jsonl(train_path)[:256]

# ------------------------------------------------------------
# 3.  Evaluation loop
# ------------------------------------------------------------
total, correct = 0, 0
all_losses     = []          # optional – to compare prompt quality

# Build demo pool (everything except current query)

for idx in tqdm(range(len(train_ds_list)), desc="Baseline eval"):
    item       = train_ds_list[idx]
    Q_inf, A_gt, A_ans = item["problem"], item["solution"], item["answer"]
    demos = [(d["problem"], d["solution"], d["answer"])
                for j, d in enumerate(raw_demos) if j != idx]
        
    # Encode query + candidate demos
    with torch.no_grad():
        q_emb     = encoder.encode([Q_inf], detach=True).squeeze(0)
        demo_embs = encoder.encode([q for (q,_, _) in demos], detach=True)

    # ------- ORIGINAL cosine retrieval -------------------------
    top_k, _ = rc.retrieve_top_k_cosine(
        q_emb, demo_embs, k=min(K, len(demos))
    )
    print(f"🔍 Top-K Indices: {top_k}")
    selected_demos = [demos[i] for i in top_k]          # length = 2

    # ------- Run the ICL model --------------------------------
    responses_nested = sample_responses_per_demo(
        demo_tuples = selected_demos,
        Q_inf       = Q_inf,
        icl_model   = icl_model,
        num_samples = NUM_SAMPLES,
        parallel=True
    )
    flat_responses = [r for demo_resps in responses_nested for r in demo_resps]
    print("Solution: ", A_ans)
    print("Response: ", flat_responses[0])
    acc = compute_demo_accuracy(flat_responses, A_ans)
    print(acc) 
    correct += acc
    total   += 1

baseline_acc = correct / total
print(f"\n📊  Baseline accuracy on MATH500: {baseline_acc:.3%}")


Baseline eval:   0%|          | 0/40 [00:00<?, ?it/s]

🔍 Top-K Indices: [149]
[INFO] Launching 1 parallel inference tasks...


Baseline eval:   2%|▎         | 1/40 [00:02<01:46,  2.73s/it]

Solution:  2
Response:  2
['2']
1.0
🔍 Top-K Indices: [104]
[INFO] Launching 1 parallel inference tasks...


Baseline eval:   5%|▌         | 2/40 [00:04<01:32,  2.44s/it]

...

Solution:  20
Response:  20
['20']
1.0
🔍 Top-K Indices: [26]
[INFO] Launching 1 parallel inference tasks...


Baseline eval:  98%|█████████▊| 39/40 [03:40<00:02,  2.62s/it]

Solution:  400
Response:  400
['400']
1.0
🔍 Top-K Indices: [96]
[INFO] Launching 1 parallel inference tasks...


Baseline eval: 100%|██████████| 40/40 [03:42<00:00,  5.57s/it]

Solution:  2
Response:  2
['2']
1.0

📊  Baseline accuracy on MATH500: 75.000%


In [ ]:
import helpers
train_path = "./data/MATH/train.jsonl"
train_ds_list = helpers.load_jsonl(train_path)
print(train_ds_list[0])

{'problem': 'How many vertical asymptotes does the graph of $y=\\frac{2}{x^2+x-6}$ have?', 'solution': 'The denominator of the rational function factors into $x^2+x-6=(x-2)(x+3)$. Since the numerator is always nonzero, there is a vertical asymptote whenever the denominator is $0$, which occurs for $x = 2$ and $x = -3$.  Therefore, the graph has $\\boxed{2}$ vertical asymptotes.', 'answer': '2', 'subject': 'Algebra', 'level': 3, 'unique_id': 'test/algebra/1.json'}


# TRAINING LOOP

In [ ]:
from tqdm import tqdm
import torch
import helpers
from mathbert_encoder import MathBERTEncoder
import retriever_cosine as rc
# from import retrieve_top_k_cosine, retrieve_sample_k_cosine
from response_sampler import sample_responses_per_demo
from reward_aggregator import compute_demo_accuracy
from icl_model_wrapper import OpenAIICLModel, OpenAIAdvanced
from grpo_optimizer import grpo_step
from datasets import load_dataset
from dotenv import load_dotenv
import os
from transformers import get_linear_schedule_with_warmup
from importlib import reload

reload(rc)

load_dotenv()

# === Settings ===
API_KEY = os.getenv("OPENAI_API_KEY")
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
K = 4
NUM_SAMPLES_PER_DEMO = 3
LEARNING_RATE =  1e-5
MAX_STEPS = 3
TEMPERATURE = 0.7

# === Init ===
encoder = MathBERTEncoder(device=DEVICE, trainable=True)
encoder.train()

icl_model = OpenAIAdvanced(api_key=API_KEY,
                           model_name="gpt-4.1-nano",
                           temperature=TEMPERATURE)
optimizer = torch.optim.Adam(encoder.parameters(), lr=LEARNING_RATE)

train_path = "./data/MATH/train.jsonl"
train_ds_list = helpers.load_jsonl(train_path)[:40]
raw_demos = helpers.load_jsonl(train_path)[:128]

# === Training Loop ===
reward_history = []
N = len(train_ds_list)
active_indices = list(range(N))

for step in range(MAX_STEPS):
    print(f"\n=== Training Step {step+1} ===")
    epoch_rewards = [None] * N                # default to None for every example
    next_active = []

    # loop only over the “hard” examples
    for idx in tqdm(active_indices, desc="Examples"):
        item = train_ds_list[idx]
        Q_inf, A_gt, A_ans = item["problem"], item["solution"], item["answer"]

        # --- encode & retrieve as before ---
        q_emb = encoder.encode([Q_inf], detach=False).squeeze(0)
        demo_questions = [d["problem"] for d in raw_demos if d is not item]
        demo_embs = encoder.batched_encode(demo_questions, batch_size=16, detach=False)
        top_k_inds, sims = rc.retrieve_sample_k_cosine(q_emb, demo_embs, k=min(K, len(demo_questions)))
        print("-------Question---------")
        print(Q_inf)
        print(f"\n🧠 Inference Index {idx}")
        print(f"🔍 Top-K Indices: {top_k_inds}")
        selected = [raw_demos[i] for i in top_k_inds]

        # --- sample responses & compute mean reward ---
        responses_nested = sample_responses_per_demo(
            demo_tuples=[(d["problem"], d["solution"], d["answer"]) for d in selected],
            Q_inf=Q_inf,
            icl_model=icl_model,
            num_samples=NUM_SAMPLES_PER_DEMO,
            parallel=True
        )
        rewards = []
        print("Responses: ", responses_nested)
        for i, responses in enumerate(responses_nested):
            reward = compute_demo_accuracy(responses, A_ans)
            rewards.append(reward)
            # print(f"    Demo {i} | Reward: {reward:.2f}")
        
        rewards = torch.tensor(rewards, dtype=torch.float32).to(DEVICE)

        loss = grpo_step(
            rewards,
            sims,
            q_emb,
            demo_embs,
            optimizer
        )

        mean_r = rewards.mean().item()
        epoch_rewards[idx] = mean_r

        # decide whether to keep for next epoch
        if mean_r <= 0.75:
            next_active.append(idx)

        print(f"✅  idx={idx} | Mean Reward: {mean_r:.4f} | Loss: {loss:.4f}")

    # record this epoch’s full reward vector (with Nones)
    reward_history.append(epoch_rewards)

    dropped = len(active_indices) - len(next_active)
    print(f"Epoch {step+1}: dropped {dropped} examples (mean_r > 0.75).")
    active_indices = next_active

    if not active_indices:
        print("All examples have converged—stopping early.")
        break




=== Training Step 1 ===


Examples:   0%|          | 0/40 [00:00<?, ?it/s]

-------Question---------
How many vertical asymptotes does the graph of $y=\frac{2}{x^2+x-6}$ have?

🧠 Inference Index 0
🔍 Top-K Indices: tensor([ 16, 113,  75,  59], device='cuda:0')
[INFO] Launching 12 parallel inference tasks...
Rewards:  tensor([1.0000, 0.6667, 1.0000, 0.6667], device='cuda:0')
Advantages:  tensor([ 0.8660, -0.8660,  0.8660, -0.8660], device='cuda:0')


Examples:   2%|▎         | 1/40 [00:04<03:00,  4.63s/it]

✅  idx=0 | Mean Reward: 0.8333 | Loss: -0.7791
-------Question---------
If $5x - 3 = 12$, what is the value of $5x + 3$?

🧠 Inference Index 1
🔍 Top-K Indices: tensor([ 92,  20, 108, 117], device='cuda:0')
[INFO] Launching 12 parallel inference tasks...
Rewards:  tensor([1., 1., 1., 1.], device='cuda:0')
Advantages:  tensor([0., 0., 0., 0.], device='cuda:0')


...

✅  idx=11 | Mean Reward: 0.2500 | Loss: -2.0731
-------Question---------
One morning each member of Angela's family drank an 8-ounce mixture of coffee with milk. The amounts of coffee and milk varied from cup to cup, but were never zero. Angela drank a quarter of the total amount of milk and a sixth of the total amount of coffee. How many people are in the family?

🧠 Inference Index 12
🔍 Top-K Indices: tensor([12, 40, 50, 79], device='cuda:0')
[INFO] Launching 12 parallel inference tasks...
Error making OpenAI API call: Could not parse response content as the length limit was reached - CompletionUsage(completion_tokens=2048, prompt_tokens=330, total_tokens=2378, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))
False
Error making OpenAI API call: Could not parse response content as the length limit was reached - Com

Examples:  46%|████▌     | 6/13 [01:17<01:25, 12.16s/it]

✅  idx=12 | Mean Reward: 0.5833 | Loss: 1.0644
-------Question---------
Lizzy, Megan, Oscar, and Patrick each have $x$ pieces of candy, where $x$ is a positive integer. Unfortunately, Patrick is the only one of the four who likes candy. So Lizzy gives all her candy to Megan. Then Megan gives all the candy she now has (which includes the candy Lizzy gave her) to Oscar. Then Oscar gives all the candy he now has to Patrick.

Let $P$ be the number of pieces of candy Patrick has in the end. How many of the following statements are true? (Assume that we do not know exactly what $x$ is.)

(a) $2$ can be a divisor of $P$.
(b) $2$ must be a divisor of $P$.
(c) $3$ can be a divisor of $P$.
(d) $3$ must be a divisor of $P$.
(e) $4$ can be a divisor of $P$.
(f) $4$ must be a divisor of $P$.

🧠 Inference Index 14
🔍 Top-K Indices: tensor([14,  0, 77, 71], device='cuda:0')
[INFO] Launching 12 parallel inference tasks...
Rewards:  tensor([0.6667, 0.0000, 0.0000, 0.6667], device='cuda:0')
Advantages:  

Examples:  54%|█████▍    | 7/13 [01:24<01:02, 10.37s/it]

✅  idx=14 | Mean Reward: 0.3333 | Loss: -1.5363
-------Question---------
The real number $x$ satisfies
\[3x + \frac{1}{2x} = 3.\]Find
\[64x^6 + \frac{1}{729x^6}.\]

🧠 Inference Index 15
🔍 Top-K Indices: tensor([15, 99, 61, 96], device='cuda:0')
[INFO] Launching 12 parallel inference tasks...
Error making OpenAI API call: Could not parse response content as the length limit was reached - CompletionUsage(completion_tokens=2048, prompt_tokens=586, total_tokens=2634, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))
False
Error making OpenAI API call: Could not parse response content as the length limit was reached - CompletionUsage(completion_tokens=2048, prompt_tokens=586, total_tokens=2634, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_pre

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# --- reshape rewards ---
# epoch_wide_rewards  -> shape (num_epochs, num_questions)
epoch_wide_rewards = [[item.item() for item in rewards] for rewards in reward_history]
reward_matrix = np.array(epoch_wide_rewards)       # (E, Q)
num_epochs, num_qs = reward_matrix.shape

# --- plot ---
plt.figure(figsize=(10, 6))
x = np.arange(1, num_epochs + 1)

for q in range(num_qs):
    plt.plot(x, reward_matrix[:, q], label=f"Q{q}", alpha=0.8)

plt.title("Per-question reward trajectory")
plt.xlabel("Epoch")
plt.ylabel("Mean reward")
plt.xticks(x)
plt.grid(alpha=0.3)

# put the legend outside so the graph isn’t cluttered
plt.legend(
    bbox_to_anchor=(1.05, 1), loc="upper left",
    ncol=2, fontsize="small", frameon=False
)

plt.tight_layout()
plt.show()

In [ ]:
# ------------------------------------------------------------
# 0.  Imports & config – nothing here should clash with yours
# ------------------------------------------------------------
from tqdm import tqdm
import torch
from mathbert_encoder import MathBERTEncoder
import retriever_cosine as rc
# from import retrieve_top_k_cosine, retrieve_sample_k_cosine
from response_sampler import sample_responses_per_demo
from reward_aggregator import compute_demo_accuracy
from icl_model_wrapper import OpenAIICLModel
from grpo_optimizer import grpo_step
from datasets import load_dataset
from dotenv import load_dotenv
import os
from transformers import get_linear_schedule_with_warmup
from importlib import reload

reload(rc)

load_dotenv()

API_KEY = os.getenv("OPENAI_API_KEY")
DEVICE  = "cuda" if torch.cuda.is_available() else "cpu"
K       = 3                # demos per query
NUM_SAMPLES = 5             # model completions per query
TEMPERATURE = 0.7           # keep same as training loop

# ------------------------------------------------------------
# 1.  Initialise encoder **in eval mode** (weights frozen)
# ------------------------------------------------------------
encoder.eval()                                   # no grads!

icl_model = OpenAIICLModel(api_key=API_KEY,
                           model_name="gpt-4.1-nano",
                           temperature=TEMPERATURE)

# ------------------------------------------------------------
# 2.  Load/define the inference set
#     If you already saved a slice elsewhere, just load it.
# ------------------------------------------------------------
gsm8k_data = load_dataset("gsm8k", "main")["train"].select(range(256))  # slice first 200 examples
gsm8k_to_infer = load_dataset("gsm8k", "main")["train"] \
                     .select(range(20))          # ⬅ change as needed

# ------------------------------------------------------------
# 3.  Evaluation loop
# ------------------------------------------------------------
total, correct = 0, 0
all_losses     = []          # optional – to compare prompt quality

for idx in tqdm(range(len(gsm8k_to_infer)), desc="Baseline eval"):
    item       = gsm8k_to_infer[idx]
    Q_inf, A_gt = item["question"], item["answer"]

    # Build demo pool (everything except current query)
    demos = [(d["question"], d["answer"])
             for j, d in enumerate(gsm8k_data) if j != idx]

    # Encode query + candidate demos
    with torch.no_grad():
        q_emb     = encoder.encode([Q_inf], detach=True).squeeze(0)
        demo_embs = encoder.encode([q for (q, _) in demos], detach=True)

    # ------- ORIGINAL cosine retrieval -------------------------
    top_k, _ = rc.retrieve_top_k_cosine(
        q_emb, demo_embs, k=min(K, len(demos))
    )
    print(f"🔍 Top-K Indices: {top_k}")
    selected_demos = [demos[i] for i in top_k]          # length = 2

    # ------- Run the ICL model --------------------------------
    responses_nested = sample_responses_per_demo(
        demo_tuples = selected_demos,
        Q_inf       = Q_inf,
        icl_model   = icl_model,
        num_samples = NUM_SAMPLES,
        parallel=True
    )
    print(responses_nested)
    flat_responses = [r for demo_resps in responses_nested for r in demo_resps]
    acc = compute_demo_accuracy(flat_responses, A_gt)
    print(acc) 
    correct += acc
    total   += 1

baseline_acc = correct / total
print(f"\n📊  Baseline accuracy on GSM8K_to_infer: {baseline_acc:.3%}")


Baseline eval:   0%|          | 0/20 [00:00<?, ?it/s]

🔍 Top-K Indices: [78, 79, 211]
[INFO] Launching 15 parallel inference tasks...


Baseline eval:   5%|▌         | 1/20 [00:01<00:22,  1.17s/it]

[['48 + (48/2) = 48 + 24 = 72', '48 + (48 / 2) = 48 + 24 = 72', '48 + (48 ÷ 2) = 48 + 24 = 72', '48 + (48/2) = 48 + 24 = 72', '48 + (48 ÷ 2) = 48 + 24 = 72'], ['48 + 24 = 72', '72', '48 + 48/2 = 48 + 24 = 72', '48 + (48/2) = 48 + 24 = 72', '144'], ['48 + 24 = 72', '48 + (48/2) = 48 + 24 = 72', '48 + 24 = 72', '72', '72']]
0.9333333333333333
🔍 Top-K Indices: [63, 119, 81]
[INFO] Launching 15 parallel inference tasks...


Baseline eval:  10%|█         | 2/20 [00:02<00:23,  1.32s/it]

...

[['47', '53', '73', '53', '33'], ['49', '55', '61', '55', '49'], ['53', '49', '26 - 4 = 22 pink hard hats remaining\nJohn takes away 6 pink hard hats and twice as many green hard hats as the pink hard hats he took, so green hard hats taken by John = 2 * 6 = 12\nRemaining pink hard hats = 22 - 6 = 16\nRemaining green hard hats = 15 - 12 = 3\nRemaining yellow hard hats = 24 (none taken)\nTotal remaining hard hats = 16 + 3 + 24 = 43', '26 + 15 + 24 = 65\nCarl takes away 4 pink hard hats: remaining pink = 26 - 4 = 22\nJohn takes away 6 pink hard hats: remaining pink = 22 - 6 = 16\nJohn takes away twice as many green hard hats as pink hard hats he removed: 2 * 6 = 12 green hard hats\nRemaining green hard hats = 15 - 12 = 3\nTotal remaining hard hats = pink + green + yellow = 16 + 3 + 24 = 43', "26 - 4 = 22 (pink hard hats remaining after Carl's removal)  \nJohn takes away 6 pink hard hats, leaving 22 - 6 = 16 pink hard hats in the truck  \nJohn takes away twice as many green hard hats as th

Baseline eval:  95%|█████████▌| 19/20 [00:28<00:01,  1.65s/it]

[['9', '9', '9', '10', '9'], ['9', '9', '9', '9', '9'], ['9', '9', '9', '9', '9']]
0.0
🔍 Top-K Indices: [18, 119, 15]
[INFO] Launching 15 parallel inference tasks...


Baseline eval: 100%|██████████| 20/20 [00:31<00:00,  1.58s/it]

[['10', '10', '8', '9', '16'], ['8', '8', '8', '8', '8'], ['16', '16', '16', '16', '20 miles each way, so 40 miles per day for 5 days: 40*5=200 miles  \nWeekend ride: 200 miles  \nTotal miles: 200+200=400 miles  \nTime = Distance / Speed = 400 / 25 = 16 hours']]
0.4

📊  Baseline accuracy on GSM8K_to_infer: 38.333%


In [ ]:
# Save the updated MathBERT model
save_path = "./53k_q3_1024_demos_0.7_temp_40_max_steps_temp_mathbert"  # your save directory
encoder.model.save_pretrained(save_path)
encoder.tokenizer.save_pretrained(save_path)

# LOADING

# from transformers import BertTokenizer, BertModel

# model = BertModel.from_pretrained("./updated_mathbert")
# tokenizer = BertTokenizer.from_pretrained("./updated_mathbert")



('./53k_q3_1024_demos_0.7_temp_40_max_steps_temp_mathbert/tokenizer_config.json',
 './53k_q3_1024_demos_0.7_temp_40_max_steps_temp_mathbert/special_tokens_map.json',
 './53k_q3_1024_demos_0.7_temp_40_max_steps_temp_mathbert/vocab.txt',
 './53k_q3_1024_demos_0.7_temp_40_max_steps_temp_mathbert/added_tokens.json')